# SGPS Molecular Techniques
## Exercise 2: Differential gene expression analysis - working with miRNAs

#### Gene Expression data from Gene Expression Omnibus
+ https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE123336
+ How to get the metadata is covered in here: https://github.com/jpwhalley/2024_Biostatistics/tree/main/e2_Getting_Data

#### Specifically from this paper
1.	LaRocca D, Barns S, Hicks SD, Brindle A et al. Comparison of serum and saliva miRNAs for identification and characterization of mTBI in adult mixed martial arts fighters. *PLoS One* 2019;14(1):e0207785.


In [ ]:
# Import the packages we need
import pandas as pd
import numpy as np
# from adjustText import adjust_text
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='poster')

In [ ]:
# Get the data in
human_expression = pd.read_csv('GSE123336_MMA_CountMatrix.csv.gz', index_col=0, compression='gzip')
human_expression = human_expression.apply(pd.to_numeric, errors='coerce')
human_expression

In [ ]:
metadata = pd.read_csv('GSE123336_metadata.csv', index_col=0)
metadata

In [ ]:
# Can we see which miRNA are differentially expressed in Serum pre and post fight?
# Filter for serum samples only
serum_samples = metadata[(metadata['tissue'] == 'Serum') & 
                          (metadata['timepoint'].isin(['0d pre', '0d post']))]

# Get the corresponding sample names
serum_sample_names = serum_samples.index

# Subset the expression data
serum_expression = human_expression[serum_sample_names]
serum_expression

pre_fight_samples = serum_samples[serum_samples['timepoint'] == '0d pre'].index
post_fight_samples = serum_samples[serum_samples['timepoint'] == '0d post'].index

pre_fight_expression = serum_expression[pre_fight_samples]
post_fight_expression = serum_expression[post_fight_samples]

In [ ]:
from scipy.stats import mannwhitneyu

# Initialize a list to store results
diff_expression_results = []

# Iterate through each miRNA
for miRNA in serum_expression.index:
    # Extract expression levels for this miRNA
    pre_values = pre_fight_expression.loc[miRNA]
    post_values = post_fight_expression.loc[miRNA]
    
    # Perform the Mann-Whitney U test
    stat, p_value = mannwhitneyu(pre_values, post_values, alternative='two-sided')
    
    # Store the results
    diff_expression_results.append({
        'miRNA': miRNA,
        'statistic': stat,
        'p_value': p_value
    })

# Convert the results to a DataFrame
diff_expression_df = pd.DataFrame(diff_expression_results)

# Apply multiple testing correction (e.g., Benjamini-Hochberg)
from statsmodels.stats.multitest import multipletests

diff_expression_df['Significant'], diff_expression_df['adjusted_p_value'], _, _ = multipletests(diff_expression_df['p_value'], method='fdr_bh')

# Sort the results by adjusted p-value
diff_expression_df = diff_expression_df.sort_values('p_value')
diff_expression_df

In [ ]:
# Calculate log2 fold change (post/pre)
log2_fold_change = (post_fight_expression.mean(axis=1) / pre_fight_expression.mean(axis=1)).apply(np.log2)

# Add fold change to the differential expression results
diff_expression_df.set_index('miRNA', inplace=True)
diff_expression_df['log2FoldChange'] = log2_fold_change

In [ ]:
# Define thresholds
cutoff_padj = 0.05  # Adjust this value if necessary
lfc_cut = 1  # Log2 fold change threshold

# Create significance flags
fdr = np.ones(len(diff_expression_df), dtype=int)
fdr[diff_expression_df['adjusted_p_value'] < cutoff_padj] = 2
fdr[(diff_expression_df['adjusted_p_value'] < cutoff_padj) & (abs(diff_expression_df['log2FoldChange']) >= lfc_cut)] = 3
diff_expression_df['fdr'] = fdr

# Plot the volcano plot
fig = plt.figure(figsize=(18, 12))
colours = ['gray', 'blue', 'red']

for i in range(1, 4):
    plt.scatter(diff_expression_df['log2FoldChange'][diff_expression_df['fdr'] == i],
                -np.log10(diff_expression_df['adjusted_p_value'][diff_expression_df['fdr'] == i]),
                s=8, c=colours[i-1])

# Add cutoff lines for log2 fold change and adjusted p-value
plt.axvline(-lfc_cut, color="k", linestyle='--')
plt.axvline(lfc_cut, color="k", linestyle='--')
plt.axhline(-np.log10(cutoff_padj), color="k", linestyle='--')

# Label the significant miRNAs
significant_miRNAs = diff_expression_df[(diff_expression_df['fdr'] == 3)]
texts = []
for label in significant_miRNAs.index:
    texts.append(plt.text(significant_miRNAs['log2FoldChange'].loc[label], 
             -np.log10(significant_miRNAs['adjusted_p_value'].loc[label]), 
             label, 
             color='black', 
             fontsize=20))
# adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=0.5))

# Add labels and title
plt.title('Volcano Plot of miRNA Expression (Pre vs Post Fight)')
plt.xlabel('log2(fold change)')
plt.ylabel('-log10(adjusted p-value)')

# Show the plot
plt.tight_layout()
plt.show()
